# Week8 Homework
## Working with Strings

The data frame contains notes taken by staff members. Your task is to clean it up to make it usable for analysis. 

### Tasks
* Create start date, end date columns and populate the values (If notes don't include month or day use 1 for missing value) 

i.e: 
    
    - 2018 => 2018-01-01  
    - June 2021 => 2021-06-01  
    - 2020 => 2020-01-01

* Create phone number column (If notes don't include phone numbers, use None)
* Create a column `duration` showing difference between start & end dates in years as a whole number

Note: Choose one of the following datasets to work on based on your taste! 

In [1]:
import re
import datetime

In [2]:
import pandas as pd

In [3]:
spicy = 'https://raw.githubusercontent.com/msaricaumbc/DS_Data/master/strings/hw_data_spicy.csv'
less_spicy = 'https://raw.githubusercontent.com/msaricaumbc/DS_Data/master/strings/hw_data_less_spicy.csv'

In [4]:
df = pd.read_csv(spicy, index_col=0)

In [5]:
df.head()

,email,notes
0,ayalajodi@hotmail.com,start date 1982-05-20 end dt 2016
1,tmitchell@hotmail.com,P# 896.729.1768x061-STDT 2004-06-12-end date 2...
2,johnrivera@gmail.com,st date:1976/end dt 05/16/2019/pn: (895)351-43...
3,kimberly60@yahoo.com,stdt1995-03 end date December 2021 pn: 881139...
4,paulharding@jefferson-burns.com,pn: (448)006-0055x240-STDT 1983-02-ed date:08/...


### Everything in lower case

In [6]:
note=df['notes']\
    .str\
    .lower()

In [7]:
for i in note:
    print(i)

start date 1982-05-20 end dt 2016
p# 896.729.1768x061-stdt 2004-06-12-end date 2012-05-27
st date:1976/end dt 05/16/2019/pn: (895)351-4305x228
stdt1995-03 end date december 2021 pn:  8811396378
pn: (448)006-0055x240-stdt 1983-02-ed date:08/29/2018
start date 05/10/1992/end date 2016-10-25/pn: 257.529.1680x3192
p/n  001-401-945-3642x69227-start date 1994-09-20-end dt 2012-06-14
pn:  695-504-7209x75874 start date october 1972 endt june 2016
stdt 02/18/1996 ed date:2018-04
stdt2003-05-24 endt 2012-09
pn: (306)435-1701x497/start date 09/08/2000/end dt 2015-12
p/n  001-858-043-5309x99644 st date:08/01/1973 end dt 2011
st date:10/07-end dt 2018-pn:  001-052-420-7633x2737
stdt 09/97/end date 2021
p#  001-083-607-9097x7171-start date 2002-09-ed date:08/20
p#  +1-097-706-8440x3521 stdt2002-01-03 ed date:2011-07
start date 2002-03-endt 2019
st date:1974-ed date:08/16-p# 217-058-2048x8261
p#  946.551.0640/st date:03/09/endt 2010
stdt08/18/1987-ed date:05/02/2012-pn: 279-081-6982x6182
p/n  (819)62

### Month List for reference in detecing dates

In [8]:
months_ls = ['january',
             'february',
             'march',
             'april',
             'may',
             'june',
             'july',
             'august',
             'september',
             'october',
             'november',
             'december',
            ]

### Start Date

In [9]:
# For Start Date:

pattern = re.compile(r'((stdt[ ])|((start|st)[ ]date)|(stdt))(.*?)([- \/]e)')

pattern_yr = re.compile(r'\d\d\d\d')
pattern_lastdigits = re.compile(r'\d\d(?!.*\d\d)')

pattern_month = re.compile(r'[:t -\/]\d\d[ -\/]')
pattern_2digits = re.compile(r'\d\d')

pattern_date1 = re.compile(r'(\d{2}[-\/ ])?(\d{2})?[-\/ ]?(\d{4})[-\/ ]?(\d{2}[-\/])?(\d{2})?|(\d{2})[-\/](\d{2})?[-\/ ]')

start_date=[]


for i in note:
    extract = pattern.search(i)
    ext_grp = extract.group()
#     print(ext_grp)
    
    #year extraction
    ext_yr = pattern_yr.search(ext_grp)

    if ext_yr == None:
        ext_yr = pattern_lastdigits.search(ext_grp)
        ext_yr = ext_yr.group()
        if int(ext_yr) < 22:
            ext_yr = int('20'+ext_yr)
        else:
            ext_yr = int('19'+ext_yr)
    else:
        ext_yr = ext_yr.group()
        ext_yr = int(ext_yr)

#     print(ext_yr)
    year = ext_yr
    
    
    #Month extraction
    month=None
    ext_mn = pattern_month.search(ext_grp)
    if ext_mn != None:
        ext_mn=ext_mn.group()
        ext_mn_dg = pattern_2digits.search(ext_mn)
        month = int(ext_mn_dg.group())
       
    else:
        for m in months_ls:
            if m in ext_grp:
                month = months_ls.index(m)+1 
                
        if month == None:
            month = 1
    
    #month is the final month in int
    
    # Date extraction
    ext_dt = pattern_date1.search(ext_grp)
    ext_dt = ext_dt.group()
#     print(ext_dt)
    if len(ext_dt)<8:
        day =1
#         print('this is 1', ext_dt)
    else:
        temp = re.compile('[-\/]\d{2}').search(ext_dt)
        temp=temp.group()\
              .strip('/-')
        day=int(temp)
#     print(date)
        
    
    #Creating a date format
    start_date.append(datetime.date(year, month, day))
    

In [10]:
df['start_date']=start_date

In [11]:
df.head()

,email,notes,start_date
0,ayalajodi@hotmail.com,start date 1982-05-20 end dt 2016,1982-05-05
1,tmitchell@hotmail.com,P# 896.729.1768x061-STDT 2004-06-12-end date 2...,2004-06-06
2,johnrivera@gmail.com,st date:1976/end dt 05/16/2019/pn: (895)351-43...,1976-01-01
3,kimberly60@yahoo.com,stdt1995-03 end date December 2021 pn: 881139...,1995-03-01
4,paulharding@jefferson-burns.com,pn: (448)006-0055x240-STDT 1983-02-ed date:08/...,1983-02-02


### End Dates

In [12]:
# End Date:

pattern = re.compile(r'(ed|end)[ ]?[t]?(date)?(.*)')

pattern_yr = re.compile(r'\d\d\d\d')
pattern_lastdigits = re.compile(r'\d\d(?!.*\d\d)')

pattern_month = re.compile(r'[:t -\/]\d\d[ -\/]')
pattern_2digits = re.compile(r'\d\d')

pattern_date1 = re.compile(r'(\d{2}[-\/ ])?(\d{2})?[-\/ ]?(\d{4})[-\/ ]?(\d{2}[-\/])?(\d{2})?|(\d{2})[-\/](\d{2})?[-\/ ]')

end_date=[]


for i in note:
    extract = pattern.search(i)
#     print(extract)
    ext_grp = extract.group()
#     print(ext_grp)
    
    #year extraction
    ext_yr = pattern_yr.search(ext_grp)

    if ext_yr == None:
        ext_yr = pattern_lastdigits.search(ext_grp)
        ext_yr = ext_yr.group()
        if int(ext_yr) < 22:
            ext_yr = int('20'+ext_yr)
        else:
            ext_yr = int('19'+ext_yr)
    else:
        ext_yr = ext_yr.group()
        ext_yr = int(ext_yr)

#     print(ext_yr)
    year = ext_yr
    
    
    #Month extraction
    month=None
    ext_mn = pattern_month.search(ext_grp)
    if ext_mn != None:
        ext_mn=ext_mn.group()
        ext_mn_dg = pattern_2digits.search(ext_mn)
        month = int(ext_mn_dg.group())
       
    else:
        for m in months_ls:
            if m in ext_grp:
                month = months_ls.index(m)+1 
                
        if month == None:
            month = 1
#     print(month)
    #month is the final month in int
    
    # Date extraction
    ext_day= re.compile(r'\d{2}[\/-]\d{2}[-\/]\d{4}|\d{4}[-]\d{2}[-]\d{2}').search(ext_grp)
    if ext_day == None:
        day =1
#         print('Here Day is 1')
    else:
#         print(ext_day.group())
        temp = re.compile(r'[\/]\d{2}[\/]').search(ext_day.group())
        if temp== None:
            temp = re.compile(r'[-]\d{2}$').search(ext_day.group())
            
        temp = temp.group().strip('/-')
        day = int(temp)
        
    
    #Creating a date format
    end_date.append(datetime.date(year, month, day))

In [13]:
df['end_date']=end_date

In [14]:
df.head()

,email,notes,start_date,end_date
0,ayalajodi@hotmail.com,start date 1982-05-20 end dt 2016,1982-05-05,2016-01-01
1,tmitchell@hotmail.com,P# 896.729.1768x061-STDT 2004-06-12-end date 2...,2004-06-06,2012-05-27
2,johnrivera@gmail.com,st date:1976/end dt 05/16/2019/pn: (895)351-43...,1976-01-01,2019-05-16
3,kimberly60@yahoo.com,stdt1995-03 end date December 2021 pn: 881139...,1995-03-01,2021-12-01
4,paulharding@jefferson-burns.com,pn: (448)006-0055x240-STDT 1983-02-ed date:08/...,1983-02-02,2018-08-29


### Phone Numbers

In [15]:
#phone number
ph = []
for i in note:
    extract = re.compile(r'p[ n:#].*[s?]|p[ n:#].*').search(i)
    line=None
    if extract is not None:
        striped=extract.group()
        line = re.sub('[-s :pn#/]', ' ', striped).strip()
        if 't' in line:
            line = line[0:line.index("t")]
    ph.append(line)



In [16]:
df['phone_number']=ph

In [17]:
df.head()

,email,notes,start_date,end_date,phone_number
0,ayalajodi@hotmail.com,start date 1982-05-20 end dt 2016,1982-05-05,2016-01-01,None
1,tmitchell@hotmail.com,P# 896.729.1768x061-STDT 2004-06-12-end date 2...,2004-06-06,2012-05-27,896.729.1768x061
2,johnrivera@gmail.com,st date:1976/end dt 05/16/2019/pn: (895)351-43...,1976-01-01,2019-05-16,(895)351 4305x228
3,kimberly60@yahoo.com,stdt1995-03 end date December 2021 pn: 881139...,1995-03-01,2021-12-01,8811396378
4,paulharding@jefferson-burns.com,pn: (448)006-0055x240-STDT 1983-02-ed date:08/...,1983-02-02,2018-08-29,(448)006 0055x240


### Duration Column

In [18]:
duration=[]
for i in range(len(start_date)):
    duration.append(end_date[i].year - start_date[i].year)

In [19]:
df['duration']=duration

In [20]:
df.head()

,email,notes,start_date,end_date,phone_number,duration
0,ayalajodi@hotmail.com,start date 1982-05-20 end dt 2016,1982-05-05,2016-01-01,None,34
1,tmitchell@hotmail.com,P# 896.729.1768x061-STDT 2004-06-12-end date 2...,2004-06-06,2012-05-27,896.729.1768x061,8
2,johnrivera@gmail.com,st date:1976/end dt 05/16/2019/pn: (895)351-43...,1976-01-01,2019-05-16,(895)351 4305x228,43
3,kimberly60@yahoo.com,stdt1995-03 end date December 2021 pn: 881139...,1995-03-01,2021-12-01,8811396378,26
4,paulharding@jefferson-burns.com,pn: (448)006-0055x240-STDT 1983-02-ed date:08/...,1983-02-02,2018-08-29,(448)006 0055x240,35
